In [1]:
import sys
sys.version

'3.7.2 (default, Dec 25 2018, 03:50:46) \n[GCC 7.3.0]'

In [2]:
import torch
import torch.nn as nn
import numpy as np

In [3]:
from importlib import reload

In [4]:
import libmotivation
reload(libmotivation)
from libmotivation import *
import mkdata
reload(mkdata)
from mkdata import *

In [5]:
user_list = get_users_with_choose_difficulty()

In [6]:
user_to_difficulty_items = {}
for user in user_list:
  user_to_difficulty_items[user] = get_choose_difficulty_items_for_user(user)

In [7]:
import arrow
from dateutil import tz

#timezone_list = list(timezone_set)
#print(timezone_list)
#print(list(map(to_hours_and_minutes, timezone_list)))

def parse_timezone_offset(datestr):
  date_parts = datestr.split(' ')
  for x in date_parts:
    if 'GMT' in x:
      output = (x.replace('GMT', ''))
      #timezone_set.add(output)
      return output
  raise Exception(datestr)

def to_hours_and_minutes(offset):
  if len(offset) != 5:
    raise Exception(offset)
  sign = offset[0]
  hours = int(offset[1:3])
  minutes = int(offset[3:])
  if sign == '+':
    return hours,minutes
  if sign == '-':
    return -hours,-minutes
  raise Exception(offset)

def adjust_timestamp_to_timezone_offset(timestamp, offset):
  hours,minutes = to_hours_and_minutes(offset)
  #print(hours, minutes)
  ar = arrow.get(timestamp / 1000.0)
  return ar.shift(hours=hours, minutes=minutes)

def get_time_adjusted_for_timezone(timestamp, datestr):
  offset = parse_timezone_offset(datestr)
  return adjust_timestamp_to_timezone_offset(timestamp, offset)

#print(adjust_timestamp_to_timezone(1548789649098.0,'-0600')) # 'Tue Jan 29 2019 13:20:49 GMT-0600 (Central Standard Time)',
print(get_time_adjusted_for_timezone(1548789649098.0, 'Tue Jan 29 2019 13:20:49 GMT-0600 (Central Standard Time)'))

2019-01-29T13:20:49.098000+00:00


In [8]:


timezone_set = set()



timestamp_server = 1544629175088.0
timestamp_local = 1544629174633.0
#(timestamp_local - timestamp_server) / (1000*3600)
#arrow.get(timestamp_local / 1000)
#timestamp
#arrow.get('Wed Dec 12 2018 07:39:34 GMT-0800 (Pacific Standard Time)')
print(parse_timezone_offset('Wed Dec 12 2018 07:39:34 GMT-0800 (Pacific Standard Time)'))

-0800


In [9]:
for user in user_list:
  difficulty_items = user_to_difficulty_items[user]
  for x in difficulty_items:
    parse_timezone_offset(x['localtime'])

In [10]:
for user in user_list[1000:]:
  difficulty_items = user_to_difficulty_items[user]
  if len(difficulty_items) > 0:
    print(difficulty_items[0])
    break


{'_id': ObjectId('5c50a792f3415400158771ed'), 'url': 'https://www.amazon.com/', 'tab_id': 42, 'session_id': 0, 'is_new_session': True, 'is_preview_mode': False, 'is_suggestion_mode': False, 'is_previously_seen': False, 'type': 'impression', 'intervention': 'internal/choose_difficulty', 'userid': '15182050961fe5b6b51277ac', 'install_id': 'c5bca283a937efd54f195c8a', 'day': 1124, 'synced': 0, 'timestamp': 1548789650566.0, 'localtime': 'Tue Jan 29 2019 13:20:49 GMT-0600 (Central Standard Time)', 'itemid': '9fa7543f0b34fa152d8cd39f', 'log_major_ver': '8', 'log_minor_ver': '1', 'habitlab_version': '1.0.263', 'id': 1, 'logname': 'internal/choose_difficulty', 'timestamp_local': 1548789649098.0}


In [11]:
print(get_most_common_difficulty_for_user('8d2c9eb27dee2dc85bca705b'))

nothing


In [12]:
print(get_most_common_difficulty_overall())

nothing


In [13]:
def get_correctness_of_naive_overall_strategy(user):
  output = {'correct': 0, 'incorrect': 0}
  difficulty_counts = get_choose_difficulty_counts_for_user(user)
  most_common = get_most_common_difficulty_overall()
  if most_common == None:
    return None
  output['total'] = sum(difficulty_counts.values())
  if output['total'] == 0:
    return None
  output['correct'] = difficulty_counts.get(most_common, 0)
  output['incorrect'] = output['total'] - output['correct']
  output['accuracy'] = output['correct'] / output['total']
  return output

In [14]:
def get_correctness_of_naive_peruser_strategy(user):
  output = {'correct': 0, 'incorrect': 0}
  difficulty_counts = get_choose_difficulty_counts_for_user(user)
  most_common = get_most_common_difficulty_for_user(user)
  if most_common == None:
    return None
  output['correct'] = difficulty_counts[most_common]
  output['total'] = sum(difficulty_counts.values())
  output['incorrect'] = output['total'] - output['correct']
  output['accuracy'] = output['correct'] / output['total']
  return output

In [15]:
correctnes_of_naive_per_user = []
for user in user_list:
  correctness_info = get_correctness_of_naive_peruser_strategy(user)
  if correctness_info == None:
    continue
  accuracy = correctness_info['accuracy']
  correctnes_of_naive_per_user.append(accuracy)

print(np.median(correctnes_of_naive_per_user))
print(np.mean(correctnes_of_naive_per_user))

0.5384615384615384
0.6131964265011353


In [16]:
correctnes_of_naive_per_user = []
for user in user_list:
  correctness_info = get_correctness_of_naive_overall_strategy(user)
  if correctness_info == None:
    continue
  accuracy = correctness_info['accuracy']
  correctnes_of_naive_per_user.append(accuracy)

print(np.median(correctnes_of_naive_per_user))
print(np.mean(correctnes_of_naive_per_user))

0.3134892511221356
0.35693836155774833


In [17]:
num_correct_unnormalized = 0
num_total_unnormalized = 0
for user in user_list:
  correctness_info = get_correctness_of_naive_overall_strategy(user)
  if correctness_info == None:
    continue
  num_correct_unnormalized += correctness_info['correct']
  num_total_unnormalized += correctness_info['total']
  #accuracy = correctness_info['accuracy']
  #correctnes_of_naive_per_user.append(accuracy)

#print(np.median(correctnes_of_naive_per_user))
#print(np.mean(correctnes_of_naive_per_user_unnormalized))
print(num_correct_unnormalized / num_total_unnormalized)

0.40769401524967863


In [18]:
num_correct_unnormalized = 0
num_total_unnormalized = 0
for user in user_list:
  correctness_info = get_correctness_of_naive_peruser_strategy(user)
  if correctness_info == None:
    continue
  num_correct_unnormalized += correctness_info['correct']
  num_total_unnormalized += correctness_info['total']
  #accuracy = correctness_info['accuracy']
  #correctnes_of_naive_per_user.append(accuracy)

#print(np.median(correctnes_of_naive_per_user))
#print(np.mean(correctnes_of_naive_per_user_unnormalized))
print(num_correct_unnormalized / num_total_unnormalized)

0.5918775069821227


In [19]:
#n_features = 4 # the choice at each step. nothing, easy, medium, hard
#n_features = 5
n_features = 4 + 4 + 7 + 5 + 69 + 5 + get_num_languages()
n_categories = 4 # the number of categories we are categorizing among. nothing, easy, medium, hard


In [20]:
difficulty_to_idx = {
  'nothing': 0,
  'easy': 1,
  'medium': 2,
  'hard': 3,
}

# def difficulty_to_idx(difficulty):
#   return {
#     'nothing': 0,
#     'easy': 1,
#     'medium': 2,
#     'hard': 3,
#   }[difficulty]

In [ ]:
import json

domain_to_productivity = json.load(open('domain_to_productivity.json'))
domain_to_category = json.load(open('domain_to_category.json'))

category_list = list(set(domain_to_category.values()))

domain_to_productivity_idx = {}
domain_to_category_idx = {}
for domain,productivity in domain_to_productivity.items():
  # should we do something about domains where productivity = 0?
  domain_to_productivity_idx[domain] = productivity + 2
domain_to_category_idx = {}
for domain,category in domain_to_category.items():
  # should we do something about domains where category = ''?
  category_idx = category_list.index(category)
  domain_to_category_idx[domain] = category_idx
print(len(set(domain_to_category_idx.values())))
print(len(set(domain_to_productivity_idx.values())))

In [22]:
def url_to_domain(url):
  #url='https://www.hello.org/bye/'
  domain=url.split('//')[-1].split('/')[0]
  #print (domain)
  return domain

print(url_to_domain('https://www.hello.org/bye/'))

www.hello.org


In [23]:


def make_tensors_for_user(user):
  features = make_features_for_user(user)
  return make_tensors_from_features(features)

# def make_tensor_from_prior_difficulties_list(prior_difficulties):
#   tensor = torch.zeros(len(prior_difficulties), 1, n_features)
#   for idx,difficulty in enumerate(prior_difficulties):
#     difficulty_idx = difficulty_to_idx[difficulty]
#     tensor[idx][0][difficulty_idx] = 1
#   return tensor

def make_tensor_from_chosen_difficulty(chosen_difficulty):
  difficulty_idx = difficulty_to_idx[chosen_difficulty]
  tensor = torch.tensor([difficulty_idx], dtype=torch.long)
  return tensor

def make_tensors_from_features_v1(features):
  output = []
  for feature in features:
    chosen_difficulty = feature['difficulty']
    prior_difficulties = feature['prior_difficulties']
    user = feature['user']
    category_tensor = make_tensor_from_chosen_difficulty(chosen_difficulty)
    feature_tensor = torch.zeros(len(prior_difficulties), 1, n_features) # n_features = 4 in this version
    for idx,difficulty in enumerate(prior_difficulties):
      difficulty_idx = difficulty_to_idx[difficulty]
      feature_tensor[idx][0][difficulty_idx] = 1
    #feature_tensor = make_tensor_from_prior_difficulties_list(prior_difficulties)
    output.append({'user': user, 'chosen_difficulty': chosen_difficulty, 'category': category_tensor, 'feature': feature_tensor})
  return output

def make_tensors_from_features_v2(features):
  output = []
  for feature in features:
    hour = feature['arrow_time'].hour
    chosen_difficulty = feature['difficulty']
    prior_difficulties = feature['prior_difficulties']
    user = feature['user']
    category_tensor = make_tensor_from_chosen_difficulty(chosen_difficulty)
    feature_tensor = torch.zeros(len(prior_difficulties), 1, n_features) # n_features = 1 in this version
    for idx,difficulty in enumerate(prior_difficulties):
      difficulty_idx = difficulty_to_idx[difficulty]
      feature_tensor[idx][0][0] = difficulty_idx
    output.append({'user': user, 'chosen_difficulty': chosen_difficulty, 'category': category_tensor, 'feature': feature_tensor})
  return output

def hour_to_hour_idx_4cat(hour): # hour: 0 to 24
  if 0 <= hour <= 6:
    return 0
  if 6 < hour <= 12:
    return 1
  if 12 < hour <= 18:
    return 2
  if 18 < hour <= 24:
    return 3
  raise Exception(hour)

def make_tensors_from_features_v3(features):
  output = []
  for feature in features:
    hour = feature['arrow_time'].hour
    hour_idx = hour_to_hour_idx_4cat(hour)
    weekday_idx = feature['arrow_time'].weekday()
    chosen_difficulty = feature['difficulty']
    prior_difficulties = feature['prior_difficulties']
    user = feature['user']
    category_tensor = make_tensor_from_chosen_difficulty(chosen_difficulty)
    feature_tensor = torch.zeros(len(prior_difficulties), 1, n_features) # n_features = 15 in this version
    for idx,difficulty in enumerate(prior_difficulties):
      difficulty_idx = difficulty_to_idx[difficulty]
      feature_tensor[idx][0][difficulty_idx] = 1
      feature_tensor[idx][0][4 + hour_idx] = 1
      feature_tensor[idx][0][8 + weekday_idx] = 1
    output.append({'user': user, 'chosen_difficulty': chosen_difficulty, 'category': category_tensor, 'feature': feature_tensor})
  return output

def make_tensors_from_features_v4(features):
  output = []
  for feature in features:
    url = feature['url']
    domain = url_to_domain(url)
    have_productivity_idx = False
    if domain in domain_to_productivity_idx:
      domain_productivity_idx = domain_to_productivity_idx[domain]
      have_productivity_idx = True
    else:
      domain_productivity_idx = None
    have_category_idx = False
    if domain in domain_to_category_idx:
      domain_category_idx = domain_to_productivity_idx[domain]
      have_category_idx = True
    else:
      domain_category_idx = None
    hour = feature['arrow_time'].hour
    hour_idx = hour_to_hour_idx_4cat(hour)
    weekday_idx = feature['arrow_time'].weekday()
    chosen_difficulty = feature['difficulty']
    prior_difficulties = feature['prior_difficulties']
    user = feature['user']
    category_tensor = make_tensor_from_chosen_difficulty(chosen_difficulty)
    feature_tensor = torch.zeros(len(prior_difficulties), 1, n_features) # n_features = 15 in this version
    for idx,difficulty in enumerate(prior_difficulties):
      difficulty_idx = difficulty_to_idx[difficulty]
      feature_tensor[idx][0][difficulty_idx] = 1
      feature_tensor[idx][0][4 + hour_idx] = 1
      feature_tensor[idx][0][4 + 4 + weekday_idx] = 1
      if have_productivity_idx:
        feature_tensor[idx][0][4 + 4 + 7 + domain_productivity_idx] = 1
      if have_category_idx:
        feature_tensor[idx][0][4 + 4 + 7 + 5 + domain_category_idx] = 1
    output.append({'user': user, 'chosen_difficulty': chosen_difficulty, 'category': category_tensor, 'feature': feature_tensor})
  return output

def get_domain_features_from_url(url):
  domain = url_to_domain(url)
  have_productivity_idx = False
  if domain in domain_to_productivity_idx:
    domain_productivity_idx = domain_to_productivity_idx[domain]
    have_productivity_idx = True
  else:
    domain_productivity_idx = None
  have_category_idx = False
  if domain in domain_to_category_idx:
    domain_category_idx = domain_to_productivity_idx[domain]
    have_category_idx = True
  else:
    domain_category_idx = None
  return domain_productivity_idx,have_productivity_idx,domain_category_idx,have_category_idx

def get_time_features_from_arrow(arrow_time):
  hour = arrow_time.hour
  hour_idx = hour_to_hour_idx_4cat(hour)
  weekday_idx = arrow_time.weekday()
  return hour_idx,weekday_idx

def make_tensors_from_features_v5(features): # this one cheats because we have the reference information included
  output = []
  for feature in features:
    url = feature['url']
    domain_productivity_idx,have_productivity_idx,domain_category_idx,have_category_idx = get_domain_features_from_url(url)
    hour_idx,weekday_idx = get_time_features_from_arrow(feature['arrow_time'])
    chosen_difficulty = feature['difficulty']
    prior_difficulties = feature['prior_difficulties']
    user = feature['user']
    category_tensor = make_tensor_from_chosen_difficulty(chosen_difficulty)
    feature_tensor = torch.zeros(len(prior_difficulties)+1, 1, n_features) # n_features = 15 in this version
    # features for current timestep
    idx = len(prior_difficulties)
    difficulty = feature['difficulty']
    difficulty_idx = difficulty_to_idx[difficulty]
    #if len(prior_difficulties) > 0:
    #  feature_tensor[idx][0][difficulty_to_idx[prior_difficulties[-1]]] = 1
    #feature_tensor[idx][0][difficulty_idx] = 1 # this is an impossible feature. see whether it learns corectly
    feature_tensor[idx][0][4 + hour_idx] = 1
    feature_tensor[idx][0][4 + 4 + weekday_idx] = 1
    if have_productivity_idx:
      feature_tensor[idx][0][4 + 4 + 7 + domain_productivity_idx] = 1
    if have_category_idx:
      feature_tensor[idx][0][4 + 4 + 7 + 5 + domain_category_idx] = 1
    # features for previous timesteps
    for idx,difficulty in enumerate(prior_difficulties):
      difficulty_idx = difficulty_to_idx[difficulty]
      #feature_tensor[idx][0][difficulty_idx] = 1
      hour_idx,weekday_idx = get_time_features_from_arrow(feature['prior_arrow_times'][idx])
      domain_productivity_idx,have_productivity_idx,domain_category_idx,have_category_idx = get_domain_features_from_url(feature['prior_urls'][idx])
      feature_tensor[idx][0][4 + hour_idx] = 1
      feature_tensor[idx][0][4 + 4 + weekday_idx] = 1
      if have_productivity_idx:
        feature_tensor[idx][0][4 + 4 + 7 + domain_productivity_idx] = 1
      if have_category_idx:
        feature_tensor[idx][0][4 + 4 + 7 + 5 + domain_category_idx] = 1
    output.append({'user': user, 'chosen_difficulty': chosen_difficulty, 'category': category_tensor, 'feature': feature_tensor})
  return output

def make_tensors_from_features_v6(features): # this one cheats because we have the reference information included
  output = []
  for feature in features:
    url = feature['url']
    domain_productivity_idx,have_productivity_idx,domain_category_idx,have_category_idx = get_domain_features_from_url(url)
    hour_idx,weekday_idx = get_time_features_from_arrow(feature['arrow_time'])
    chosen_difficulty = feature['difficulty']
    language_indexes = convert_language_list_to_language_indexes(feature['languages'])
    prior_difficulties = feature['prior_difficulties']
    user = feature['user']
    category_tensor = make_tensor_from_chosen_difficulty(chosen_difficulty)
    feature_tensor = torch.zeros(len(prior_difficulties)+1, 1, n_features) # n_features = 15 in this version
    # features for current timestep
    idx = len(prior_difficulties)
    difficulty = feature['difficulty']
    difficulty_idx = difficulty_to_idx[difficulty]
    #if len(prior_difficulties) > 0:
    #  feature_tensor[idx][0][difficulty_to_idx[prior_difficulties[-1]]] = 1
    #feature_tensor[idx][0][difficulty_idx] = 1 # this is an impossible feature. see whether it learns corectly
    feature_tensor[idx][0][4 + 4 + 7 + 5 + 69 + difficulty_to_idx[feature['initial_difficulty']]] = 1
    for language_index in language_indexes:
      feature_tensor[idx][0][4 + 4 + 7 + 5 + 69 + 5 + language_index] = 1
    feature_tensor[idx][0][4 + hour_idx] = 1
    feature_tensor[idx][0][4 + 4 + weekday_idx] = 1
    if have_productivity_idx:
      feature_tensor[idx][0][4 + 4 + 7 + domain_productivity_idx] = 1
    if have_category_idx:
      feature_tensor[idx][0][4 + 4 + 7 + 5 + domain_category_idx] = 1
    # features for previous timesteps
    for idx,difficulty in enumerate(prior_difficulties):
      difficulty_idx = difficulty_to_idx[difficulty]
      #feature_tensor[idx][0][difficulty_idx] = 1
      hour_idx,weekday_idx = get_time_features_from_arrow(feature['prior_arrow_times'][idx])
      domain_productivity_idx,have_productivity_idx,domain_category_idx,have_category_idx = get_domain_features_from_url(feature['prior_urls'][idx])
      feature_tensor[idx][0][4 + hour_idx] = 1
      feature_tensor[idx][0][4 + 4 + weekday_idx] = 1
      if have_productivity_idx:
        feature_tensor[idx][0][4 + 4 + 7 + domain_productivity_idx] = 1
      if have_category_idx:
        feature_tensor[idx][0][4 + 4 + 7 + 5 + domain_category_idx] = 1
      feature_tensor[idx][0][4 + 4 + 7 + 5 + 69 + difficulty_to_idx[feature['initial_difficulty']]] = 1
      for language_index in language_indexes:
        feature_tensor[idx][0][4 + 4 + 7 + 5 + 69 + 5 + language_index] = 1
    output.append({'user': user, 'chosen_difficulty': chosen_difficulty, 'category': category_tensor, 'feature': feature_tensor})
  return output

make_tensors_from_features = make_tensors_from_features_v6

history_length = 10

def make_features_for_user(user):
  output = []
  difficulty_items = get_choose_difficulty_items_for_user(user)
  prior_difficulties = []
  prior_urls = []
  prior_arrow_times = []
  languages = get_languages_for_user(user)
  initial_difficulty = get_initial_difficulty_for_user(user)
  if initial_difficulty == None:
    return []
  for item in difficulty_items:
    if 'is_random' in item and item['is_random'] == True:
      continue
    if 'type' not in item:
      continue
    if item['type'] != 'action':
      continue
    if 'difficulty' not in item:
      continue
    difficulty = item['difficulty']
    url = item['url']
    arrow_time = get_time_adjusted_for_timezone(item['timestamp_local'], item['localtime'])
    output.append({'url': url, 'user': user, 'initial_difficulty': initial_difficulty, 'languages': languages, 'difficulty': difficulty, 'arrow_time': arrow_time, 'prior_urls': prior_urls[:], 'prior_difficulties': prior_difficulties[:], 'prior_arrow_times': prior_arrow_times[:]})
    prior_difficulties.append(difficulty)
    prior_urls.append(url)
    prior_arrow_times.append(arrow_time)
    if len(prior_difficulties) > history_length:
      prior_difficulties = prior_difficulties[-history_length:]
      prior_urls = prior_urls[-history_length:]
      prior_arrow_times  = prior_arrow_times[-history_length:]
  return output


In [24]:
print(arrow.get().shift(hours=-8, days=-4).weekday())

4


In [25]:
#print(make_tensors_from_features([{'url': 'facebook.com', 'user': 'foobar', 'difficulty': 'easy', 'prior_difficulties': [], 'arrow_time': arrow.get().shift(hours=-8)}]))
#print(make_tensors_from_features([{'url': 'facebook.com', 'user': 'foobar', 'difficulty': 'easy', 'prior_difficulties': ['easy', 'nothing'], 'arrow_time': arrow.get().shift(hours=-8)}]))


In [26]:
all_data = []
for user in user_list:
  print(user)
  tensors = make_tensors_for_user(user)
  for tensor in tensors:
    all_data.append(tensor)

e0ea34c81d4b50cddc7bd752
d45b8b82dec8ec3c591fdbd7
f4d95006c663a799d9185576
8d2c9eb27dee2dc85bca705b
4a5f492c987b873871be4c12
78cc8c70b8be7412e00a1575
04c7fb2ae3098eab5ae390b8
8642884bd204eafc660a04a2
9ca53f1f33ac3e25c2e3f3d2
6e055258e3625ab1db57f0fa
9c00827409f8c296d67f78b4
687606ae6354d8b707f064ee
57ddb2820db2de361757d21b
af8454409cde73bead9d1071
eccfe39b9019a3f566346399
f32cf25130f6fa751892960e
8f77439cf19de38abfd4a417
3bc557eda64de5ff9d2e6fca
16043b6d098957d10448c009
edc3437cabf565d7a8c2f96c
88550c3144f8e1a6e84e83e2
3bc201cd34fc67185861245e
472a909dcf35ed6f1bc6da82
f45c319882842376492cffa7
150f93e5b7dea556c943813e
93b71831840fb44e554b6e36
cb266f371364b060f117b6a1
4e69f3aebac131196a8f2912
834ab49ae4351c22a1d49c2d
43a6fdadccd61a51caf5b588
968e70f748b83408bf3a9fa3
f6e9666fa6fd68a8ff0e2781
48df425ff6b2910b961070e5
ffdc2b2d686dbf9c8c621b35
42564d59c50078ce37d551e1
9595707af25dc9cf08d162f2
414492408538c282663be78d
8880741c0eb16b95ba062848
cc43e1c331c9d74bf7b00fc6
ae97c36156bb44a178a6519d


34329b196b73616e36d96772
a208d0a59ec5856af2dc50e1
4a32760f173ef7ace6047541
d84a3bc360e706c13c38e6bf
8617b058a51286a5ab7b9e1c
8da35e44d84233ec248f4172
33a883006ed5d3235f00978b
928c527e00506a4158cdccfb
b05febd5854b233d269b04e2
71098d5b5fa2c43aca67e72c
b27520836efa5514d7e606a0
8c68857d77921615ec167ca3
b586773da2f5ac24536eeb4f
ce1ef1c40c2bd0dd1e2f3aa2
b120e76e564050b2d0457792
99dfe99b1f4d6fa8cdbb4110
c57b61dbfa89921bf4679013
2fb4052db10fb9e1d796c14f
44c361621083a0c056b029b4
d2a3afc469e10fac6f90a5e5
759798d48ea480cd146738d4
1310cd2f7f35173a9551c8cf
8e2da14192c115ee45870d0c
4bebe3f9d6a73dcfa0ba288e
f2ca89d4971eba6f0a2b4da7
b02ec08559176d815a86c29c
f0b3d19bb278c0334accee9f
f4777b7e5d213643f6f7be96
6584dc6e6039cf7bc6a65d78
fb9a51627e26f2af8dab5dc8
4c849873e044b007b91f7f39
90440b09cd2c17ca7784f21d
572a93aa3041afcab9c816e2
b570d0d6f46f095ccdd24376
40b957b71fcc72d41866e6b8
3056a2d989be02294adab708
0038a28ca863a4f560d099e0
63b6228a68cbc5f3597e9743
92202175520ab67835acedff
78cb68240cb3a717a5d6fa31


f32cb59bc2542875d8f95312
4447f0de5f79dc60750989f5
26ff31f781dee4d279d85c22
4f4ee561e3dd421b86d932b3
d0fd104fb17c12ed4bec135a
9d14195bae7976d99c743bde
302a6630c2a7269db7303c2e
83d1b05e8acb0fb0984e8db7
8eec8af6498d29e87323fd9e
c595e6759c26aa7f898a637d
6de6cf27c09af8b136a6a950
e3d650eb237f5dfca22f1835
0bc3e78c4ecd05a6573836a6
e1db0b6bb8cdc2a6a6879b48
319647af484aacc0d5fabc7d
2cc11117ba44aa9f0dec69a1
dd255758943abf38102497dc
243ea8620fd06631645f326a
22e9433a1d7be8a0dae6ea9c
bbf13fd8b25105f117b3c931
d1c5ef05d3adcfb188344a24
490c823ad1e5e87573af0615
5d553942942322c5fce88a96
1c499412ea5b034f2d45e0fe
a6a4b8327878e0849f128028
39142cdc607dc59a7bc1b095
f7bc103e54d34c39fe4f34bd
63246bd51af632966de41ff5
2b094a50ccfbe785e2ab83a9
c26b116cdec7c37ff5dfc612
cdc94c551609bd8a8c26b624
b83f7d5ff4c3695712f94f3b
75d2f3897e1f41b7f1a9f584
49d39fa16572b1652523d87e
85d88b9d04b7c08dd3ad0c11
8e9a3e1717d60f22873441b6
ac5db577f6903de2b1742f1a
5b30481359252552b9dde0fb
a0d5dad7a6149744236f0d12
f59bfe48cbe62d84b67a6342


51c4b8746e08a513d0f5916f
aa707648e359b70b424933e3
eb2621c317e7446ad8f3f1f4
d4d7fc6a23f0fc1c30a1cdd5
34f4f1ffb7bdd7765dfac9a0
ed906381847fef32519dca51
71aadc5ff3e384da50b1e7fd
b897d8aa5f98412ed2728c09
78408af649dd586576edab66
ffffd52cfa5f2e9bf6a10f0a
946d3f0144f49b641b23d10c
fbb257cf531138519bddf084
15182050961fe5b6b51277ac
9f2a40a13ba640089c553221
310f89cf21281b27d434c2c7
1e36759d62b51ee55127e3ea
475e889d20140acc374dc8df
9457cef4b3bde71691a23e27
0576e33e9656e70e42809000
53c0fea8a3fe5fba4ebda953
974ce09b370f04eb60cca20c
672ee80648388a96a496c67a
13fa72924b5f4e242b79ea98
e0c2d589a97e54d104aaf20f
c360bda9083464a313c3d433
29c9ecdd69f93e2fd3da2901
c9337b5331942cc77a1dcb5d
d9663de4d0124faa17426389
28ae4bcee9e60f01e2ff592a
4cbcece86b63558a939b1195
bf915c3007979242bde4c39c
2bc37f68bc25090ca616dad3
f675a200db9e395a0d9664c1
56306df6e1b7069210a9a9aa
f2b7b18eee41f36c4378d68f
b1176f621b5dbdcdc00e2603
131b8588bce3f9496dd7a766
44e1491ea993238234ffb95b
6c2584797ff3e54db977f190
e84ebcbdece5a606a5177097


ec29ce94ade655ba1a56c99b
1af680dd003843539af951e5
fe9e6cbcf603d905d24e67a4
4d2d9bf2865eb9218512c7af
729f1784d10cc5bf3c7f9f72
eddff39417479e7ab36580c1
6a89392321b4cc22897c501a
3d640c3e8dd41a9de29bb6e6
d7aa3ce5f83e32f80e4b5b11
55fad49647c2e94d9dafe0a1
089bae43e5b57391faf16c21
ffed3f5e51a6cd5dad72fc1d
9018466f8ff19644a8f0f07e
abc1eb99b84024205b221b78
3a201b104df2016873459dd2
69bd8ccbeed4b4a0e2694f92
b10035b966af5ff385c8c428
937db5bb54959eb2e657425a
096c6c6c8863bd2decbc9a5c
5a52d5095bee8b1bf9cc4587
bab3deb854ff260b056f98e8
65d63f88976a750c1a281c29
5836368d5a92492b886035ed
b34f53474d5e01068bfe475b
b7e03e9fd9642583b0942688
680fa57492ec14ee7cf2b625
ffd02244bc2124d362a88918
01a90f3b3e469b6b8674d365
312e85bec3146b4f12701d35
8cdc00dae8e09e5618bf0e6a
6289665fbced043b0b7149d5
191c2a6df4cb1537499af61e
6612fc57f78c61407f56023f
34681c7ff5b1fd15de37d9ee
becd2c45ab0d292299ea389c
1b8271ad47904753d9d1e4b3
a63d4eaa8130dc9f7cf02b28
685eda0aace01fb1b837961b
769b4a8de599c811ca3c675e
77fc8bafae1999b7a9655b12


...

0941755efb04c1833c41ec9e
9b3e2f2557a1b50c1a48e016
5f7b347a8b6d4f436f76e33d
6e6b24d45464e7d848c94d2e
7150028f22b7dcce1545c77f
2a3f235757de269e8826ca64
584d51eb8ccbe9ba2d947f10
e9acafdec315c61e6476ceab
1d75e514a2e441f4ba8fa9de
d1de9d60d492c92a8d0548fa
6d73e96a2b98d5c62e3cf046
429c133d5a3d2e2698444f6e
068ef276351ddc4550832793
50af265195f8a33d159578d0
b272876595bde4b4546469f3
caca1846a26c3205a536607e
841da2373cf67cbd30f3f733
523a1924286a3a51b6025185
faeef257cf7ba85cfb4c1d57
5d564572874fbe80234d33dd
d88de7b80abc9305bdadc8c9
cc2609a70b3e7100d8d449f3
aa7ee53878b3a9f78fbf2f06
b1742a8d4889678d1384f42d
7c7f8b9eabaea82ca6eeeb03
a6526baed9a380476de5dc49
38f168556965a4a240440483
cd306cbadf734cdfe38bb4e8
25947319ceac605a804c01fa
aa8806be689e8b4e321011ef
13b166ccc55a86efd762e766
22395c694657a4958a175fe1
ab3bdefbfc1fdf8590e02421
946f35cde670fc8da973078e
d9832a9260d18fd992f19ee1
7b8d050c7d5cc7deebf352f4
69f991c2ce004d62b418aa14
77844b1ec8de15b4f0e078a0
8d39a7266fab2424c0fc5833
e4e31fcbde75c48bbf86635e


8d06822627c898fadb6481da
88110a8afd64c8b829313910
316c7ba3bb7a5b9310c84354
d1520b2121e5f5b0c73b0e0d
cf927fbbd47010cff7880436
fbc07351cbc17e60ef7cebe1
fc40abab7f0d7e2d024d7b9b
4eebb63a9d05e72507d852f1
ae8be555fe73516b235768e4
5cb5e6c9bccb692fca3989f5
a18cffdb793fa25ea17d7f58
e3b8e797e8ea2423c5c21a6d
03ca1d45c738a4c9e01eab4a
a19ee11aa8ea4a536ddf7f30
58b889276095232a66ce2b2b
108639bc8b27425f62afbc23
d7e8d24faedb13b1b222b441
f5bda553b5e85f088d2506a7
9a7e07a4770abf58a4f10793
fbe862d6db6a570f9be2a3de
52e6db814e9c9ac473e8c7bd
bbdb8e9dc41c26e881847d2f
e92381573c8646a2e90e7b3a
4abffbb2a84f521c65427a8d
d7924de9ac5623585fcb473a
6321bbf34972d9173be9a643
dceeafbc8a0f2abdfc78c187
37d0545a56153b6f5c71e3a2
255113af3264f045d046b7cc
7d7dfd3c493d1f522e8496ff
7fac75923e120f20d6d0a203
0412345d83f8e385cd6929ea
6551da7d143f39cbbc2140be
fd2e9ef6b655dbaa3ae3240b
7675c25761b69d0bdbadd7ad
e067dcbd3402b326bffdb01e
01c6abeb634bc2c6bbafb075
628b314f2ebc56f43796e7bf
ce6854fe9ebd13bb1a5e1f4b
8946835597c2623b1cc47c34


1d1ce7e3fa7c6a00ac664253
478848f3d10cc4259a61b636
bb89fae14404295617e52a42
3ebe79debf79ed4ab5aedf37
530ba280a6fdc281b7a4ff02
661636f595149ce7b717e29c
5f21731bfe1d6a2573ab676d
ec0bb6a14dc7a35621889c29
4cd0f1469645eb3d3d7f24f5
661eabcab4e1a4f4430a8bf5
dded86b9b792ebb64ab46c5a
338d13c05d82b6304abc31ba
6f810b9b423bf4a9df184208
dcfdf3c1a077f5484273b19a
1e4b6f1a32a8afbba80d1ffb
594534a38e522e13f112e214
67bcc204b72a41923cf57b18
f00cd5386aa04294ee4f925c
80bfcbe36e50e8e0bb4a875b
6bd7353549104727511641c5
33a18045625d7ae3301f3fd0
7ab8de580da3df346560657f
9b0f9bcb485636cf37ce4138
cb7b221ccec65fccfe1c11ff
e5d0c1c55df827266497058f
5ae1f55a3508a50a15e6d96c
4ac54eb95de00d1fa29b07ac
bdf339fe8a575bac1cdaea97
7ff430262aef9a667c1159fe
bcddb9668fcdaacca055ed8f
1923bd6c561227672cb22b37
d5d7f2dd65f5accf9bead060
4ae2a9861ff1d578616544cb
2d57bde994c285390f24d9da
a368492f1603e630d1f6365e
adb0288335a333a023c52450
4160b92ee4c1ff46930e62bb
3a9f0c0706cd544445e285be
ae88d66456d8ce09ec21e961
126cd7fb3b41d60bf1daa09e


fa475b7e6a9098b2a161cd9b
8bffbe5a419e5f1204f1ed5f
57baae75d5c16a538cfb93cc
f611b80f0b9ca860ec6b1820
b8412ee1c4f463144366f8b3
222c1d5ad1aaba5c33ed7619
f5dc57a2a8c085b4872979ad
0df61722f821bd582073f8f9
f18ab8ed68f7f6fa65777f27
5bed81bfa36d9f9e93c0287b
d65e506598ccaeea141fd862
8ceda08e8b771cf2a40cf954
19ac67df02f25a13b29085fd
2bd6c21e2cf991d66d573911
3ad79a8d8657a280254670e5
02f48e0c1d3ae320da623465
aad0154a037cf20c9de4c7ce
10edd50e1e841fcb0e051859
b4f7fe98679849bcaef75089
75b774d38e0901a5a6e6f3f9
8163f2885ab9c8cb5c769f57
69d2fa811976737c138b1a66
2ea14592934ace00fa012187
8eca7903caed038376666931
8a74bba57175650f1bf47d2d
b6c0e233809affdedbdc1617
d2da63635c5399e5f85ddc2f
8f7266979011fc6a823c9660
426e99068470569a29e1878c
41e59ca1d9d0e2fb6e323c55
70cfe3b9e067c709578b428e
c6114c3baa3e79910c289d35
2d40d4f8e0890b90f4ce8db7
9a38272c8f930479d265b552
661b567a299d00f490ed1753
d2eaaff3366350067b500300
10158647498389fa30a74596
89f6f0039eb17b7e3fe1e3a1
9b97e7c6a196ecff376f6286
870137885131af2774fbdc10


591dd7c84ea9441f11e58ed4
864db48a63aa340429d217d1
a6b083ca2020fea9f9bc7bd2
b5bff94db60d4d41f05f74d4
93ee87c3a11f14c690385603
62e876511f4cc230947a5aed
52d0a57ee6b83c85154bd76d
faf4b098f989fa77bb9d98ba
c98b2f69cfd2727de58d6dbd
ed8b414e1af56dd1cb0709f3
90dc228d9e58a3f9a9ec53b1
12d9444bcc339d1d5d6fc5de
18dbd3ef5ed9a548b1998f3a
0f0490a888e603867a35058d
b99aeb4383a0542ba4d756a6
5995f77a348de0064175a326
c5356b645f23a4bbc0751168
ae4a00580581d2b4d93a3bf0
6ea26035c8f824f0a53619ec
fc748f5598e7fa70f2a43732
531bb92a09e5921e870ed289
8d6678d38431a0645fd97842
66979b40bcc2cd19e4dff574
eec923f81af818fa94401e7b
781fe2f592e670593be7e270
653dc66f74378efc4ba3e563
71125bf808ad2bd5aee15c03
5c55c8d0c056e2c9057bfc5b
430a2b6a4edca051fe0147b5
efcac82606b2fae6d2a4afd0
eec5addebf516b06f13f5d2e
ad738e7c40484945c6ed1e80
4c037ce18739e5a695310d02
75494292416b67ca253d57eb
bff65b378c7616f8dff5ebd6
555bc0d71d89c19f63150393
80691ca407ace23f457b3990
20c49fbdc9f8c5e529f2c852
244a6ae4171e411fcb0df9f6
cd3c2d5187f482b2027ef11d


In [27]:
def tensor_to_difficulty(tensor):
  difficulty_idx = tensor[0].data.cpu().numpy()
  return ['nothing', 'easy', 'medium', 'hard'][difficulty_idx]

print(make_tensor_from_chosen_difficulty('hard'))
print(tensor_to_difficulty((make_tensor_from_chosen_difficulty('hard'))))

tensor([3])
hard


In [28]:
training_data = all_data[:int(math.floor(len(all_data)*9/10))]
dev_and_test_data = all_data[len(training_data):]
dev_data = dev_and_test_data[:int(math.floor(len(dev_and_test_data)/2))]
test_data = dev_and_test_data[len(dev_data):]

def iterateTrainingData():
  output = []
  for data in training_data:
    category = tensor_to_difficulty(data['category'])
    #yield category,data['category'],data['feature']
    output.append((category,data['category'],data['feature']))
  np.random.shuffle(output)
  return output

def randomTrainingExample():
  data = random.choice(training_data)
  category = tensor_to_difficulty(data['category'])
  return category,data['category'],data['feature']

#def makeDataTensor(lines_of_data):
#  output = 

In [29]:
#print(randomTrainingExample())
category,category_tensor,line_tensor = iterateTrainingData()[4]
print(category_tensor.size())
#print('line tensor size dimension 1 is')
#print(line_tensor.size()[0])
#train(category_tensor.cuda(), line_tensor.cuda())


torch.Size([1])


In [30]:
criterion = nn.NLLLoss()


In [31]:
learning_rate = 0.005 # If you set this too high, it might explode. If too low, it might not learn

def train(category_tensor, line_tensor):
    hidden = rnn.initHidden().cuda()

    rnn.zero_grad()

    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)

    loss = criterion(output, category_tensor)
    loss.backward()

    # Add parameters' gradients to their values, multiplied by learning rate
    for p in rnn.parameters():
        if p.grad is None:
          continue
        p.data.add_(-learning_rate, p.grad.data)

    return output, loss.item()

In [3]:
torch.cuda.device_count()

1

In [33]:
device = torch.device('cuda') 

In [37]:

class RNN1(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN1, self).__init__()

        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

n_hidden = 512

class RNN2(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN2, self).__init__()

        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        print('pre softmax output size is')
        print(output.size())
        output = self.softmax(output)
        print('post softmax output size is')
        print(output.size())
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

n_hidden = 512

rnn = RNN2(n_features, n_hidden, n_categories).cuda()

class RNN3(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=2, dropout=0.2):
        super(RNN3, self).__init__()

        self.drop = nn.Dropout(dropout)
        self.nlayers = num_layers
        self.nhid = hidden_size
        self.hidden_size = hidden_size
        self.rnn = nn.LSTM(input_size, hidden_size, num_layers, dropout=dropout)
        self.decoder = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

        #self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        #self.i2o = nn.Linear(input_size + hidden_size, output_size)
        #self.softmax = nn.LogSoftmax(dim=1)
    def forward(self, input, hidden):
      #print('input.shape in forward is')
      #print(input.size())
      #print('hidden[0].shape in forward is')
      #print(hidden[0].size())
      #print('hidden[1].shape in forward is')
      #print(hidden[1].size())
      input = input.view((input.size(0), 1, input.size(1)))
      output,hidden = self.rnn(input, hidden)
      #print('pre softmax output size is')
      #print(output.size())
      output = self.decoder(output.view(output.size(0)*output.size(1), output.size(2)))
      output = self.softmax(output)
      #print('post softmax output size is')
      #print(output.size())
      return output, hidden

    def initHidden(self, bsz=1):
        return (torch.zeros(2, 1, 256).cuda(), torch.zeros(2, 1, 256).cuda())
        #weight = next(self.parameters())
        #return (weight.new_zeros(self.nlayers, bsz, self.nhid),
        #       weight.new_zeros(self.nlayers, bsz, self.nhid))
    
    #def initHidden(self):
    #    return torch.zeros(10, 2, 1, self.hidden_size)

n_hidden = 512

rnn3 = RNN3(n_features, n_hidden, n_categories).cuda()

In [38]:
# model = model.RNNModel(args.model, ntokens, args.emsize, args.nhid, args.nlayers, args.dropout, args.tied).to(device)
# LSTM 33278 200 200 2 0.2 False
# rnn_type = args.model='LSTM'
# ntoken = ntokens=33278 (number of tokens. presumably the number of words in the english language)
# ninp = args.emsize=200 (size of word embeddings)
# nhid = args.nhid=200 (number of hidden units per layer)
# nlayers = args.nlayers=2 (number of layers)
# dropout = args.dropout=0.2 (dropout applied to layers (0 = no dropout))
# tie_weights = args.tied=False (tie the word embedding and softmax weights)


import torch.nn as nn

class RNNModel(nn.Module):
    """Container module with an encoder, a recurrent module, and a decoder."""

    def __init__(self, rnn_type, ntoken, ninp, nhid, nlayers, dropout=0.5, tie_weights=False):
        super(RNNModel, self).__init__()
        #self.drop = nn.Dropout(dropout)
        self.encoder = nn.Embedding(ntoken, ninp)
        self.rnn = nn.LSTM(ninp, nhid, nlayers, dropout=dropout)
        self.decoder = nn.Linear(nhid, ntoken)
        
        self.init_weights()

        self.rnn_type = rnn_type
        self.nhid = nhid
        self.nlayers = nlayers

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, input, hidden):
        #print('input is')
        #print(input.size()) # torch.Size([35, 20]) # 35 = sequence length, 20 = batch size
        #emb = self.drop(self.encoder(input))
        #print('emb is')
        #print(emb.size()) # torch.Size([35, 20, 200]) # 35 = sequence length, 20 = batch size, 200 = embedding size
        emb = input
        output, hidden = self.rnn(emb, hidden)
        return output,hidden
        #output = self.drop(output)
        #decoded = self.decoder(output.view(output.size(0)*output.size(1), output.size(2)))
        #return decoded.view(output.size(0), output.size(1), decoded.size(1)), hidden

    def init_hidden(self, bsz):
        weight = next(self.parameters())
        return (weight.new_zeros(self.nlayers, bsz, self.nhid),
                    weight.new_zeros(self.nlayers, bsz, self.nhid))
        return (weight.new_zeros(self.nlayers, bsz, self.nhid), weight.new_zeros(self.nlayers, bsz, self.nhid))

rnn2 = RNNModel(rnn_type='LSTM', ntoken=4, ninp=4, nhid=200, nlayers=2, dropout=0, tie_weights=False)

# def makeIntoTensor(data_list):
#   output = {}
#   line_to_user = {}
  
#   for idx,item in enumerate(training_data):
#     print(idx)
#     print(item)
#     print(item.size())
#     break

# def makeTrainingData():
#   return makeIntoTensor(training_data)
  

# makeTrainingData() 

In [39]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import math, copy, time
from torch.autograd import Variable
import matplotlib.pyplot as plt

class PositionwiseFeedForward(nn.Module):
    '''
    Feed-forward layer
    '''
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.w_2(self.dropout(F.relu(self.w_1(x))))

def attention(query, key, value, mask=None, dropout=None):
    '''
    Helper function for calculating attention
    '''
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) \
             / math.sqrt(d_k)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    p_attn = F.softmax(scores, dim = -1)
    if dropout is not None:
        p_attn = dropout(p_attn)
    return torch.matmul(p_attn, value), p_attn

class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0.1):
        super(MultiHeadedAttention, self).__init__()
        assert d_model % h == 0
        # We assume d_v always equals d_k
        self.d_k = d_model // h
        self.h = h
        self.linears = clones(nn.Linear(d_model, d_model), 4)
        self.attn = None
        self.dropout = nn.Dropout(p=dropout)
        
    def forward(self, query, key, value, mask=None):
        if mask is not None:
            # Same mask applied to all h heads.
            mask = mask.unsqueeze(1)
        nbatches = query.size(0)
        
        # 1) Do all the linear projections in batch from d_model => h x d_k 
        query, key, value = \
            [l(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
             for l, x in zip(self.linears, (query, key, value))]
        
        # 2) Apply attention on all the projected vectors in batch. 
        x, self.attn = attention(query, key, value, mask=mask, 
                                 dropout=self.dropout)
        
        # 3) "Concat" using a view and apply a final linear. 
        x = x.transpose(1, 2).contiguous() \
             .view(nbatches, -1, self.h * self.d_k)
        return self.linears[-1](x)

class Encoder(nn.Module):
    def __init__(self, layer, N):
        super(Encoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)
        
    def forward(self, x, mask):
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

def clones(module, N):
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

class LayerNorm(nn.Module):
    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

class SublayerConnection(nn.Module):
    """
    A residual connection followed by a layer norm.
    Note for code simplicity the norm is first as opposed to last.
    """
    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        return x + self.dropout(sublayer(self.norm(x)))

class EncoderLayer(nn.Module):
    def __init__(self, size, self_attn, feed_forward, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 2)
        self.size = size

    def forward(self, x, mask):
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))
        return self.sublayer[1](x, self.feed_forward)

class NLPTransformer(nn.Module):
    def __init__(self, window_embed_size, embed_dim=256, h_dim=128, 
                 N=6, d_ff=128, h=8, dropout=0.1, n_layers=1,
                 device=torch.device('cuda:0')):
        super(NLPTransformer, self).__init__()
        self.embed_dim = embed_dim
        self.h_dim = h_dim
        # embedding layers
        # Create raw-to-embed FC+Dropout layer
        self.embed = nn.Sequential(nn.Dropout(0.1),
                                   nn.Linear(window_embed_size, embed_dim),
                                   nn.ReLU())
        # modality -> only linguistics -> output embed_dim

        # encoder (6 encoders)
        # encoder = encoder layer + sublayer connection
        # encoder layer = attention layer + feedforward + norm layer
        c = copy.deepcopy
        attn = MultiHeadedAttention(h, embed_dim)
        ff = PositionwiseFeedForward(embed_dim, d_ff, dropout)
        self.encoder = Encoder(EncoderLayer(embed_dim, c(attn), c(ff), dropout), N)
        # Decodes targets and LSTM hidden states
        self.decoder = nn.LSTM(1+embed_dim, embed_dim, n_layers, batch_first=True)
        self.dec_h0 = nn.Parameter(torch.zeros(n_layers, 1, embed_dim))
        self.dec_c0 = nn.Parameter(torch.zeros(n_layers, 1, embed_dim))
        # the output will be in the embed_dim dimension
        # output only 1d
        self.out = nn.Sequential(nn.Linear(embed_dim, h_dim),
                                 nn.ReLU(),
                                 nn.Linear(h_dim, 4),
                                 nn.LogSoftmax(dim=1))
        # Store module in specified device (CUDA/CPU)
        self.device = (device if torch.cuda.is_available() else
                       torch.device('cpu'))
        self.to(self.device)

    def forward(self, inputs, mask, lengths, tgt_init=0.5, target=None):
        # Get batch dim
        batch_size, seq_len = len(lengths), max(lengths)
        # Convert raw features into equal-dimensional embeddings
        embed = self.embed(inputs)
        encoder_output = self.encoder(embed, mask) # batch_size, seq_len, self.embed_dim
        # LSTM output from the encoder
        # Set initial hidden and cell states for decoder
        h0 = self.dec_h0.repeat(1, batch_size, 1)
        c0 = self.dec_c0.repeat(1, batch_size, 1)
        predicted = []
        p = torch.ones(batch_size, 1).to(self.device) * tgt_init
        # o_prev = torch.zeros(batch_size, self.embed_dim).to(self.device)
        h, c = h0, c0
        for t in range(seq_len):
            # Concatenate prediction from previous timestep to context
            i = torch.cat([p, encoder_output[:,t,:]], dim=1).unsqueeze(1)
            # Get next decoder LSTM state and output
            o, (h, c) = self.decoder(i, (h, c))
            # o_prev = o.squeeze(1)
            # Computer prediction from output state
            p = self.out(o.view(-1, self.embed_dim))
            predicted.append(p.unsqueeze(1))
            # print(predicted)
        predicted = torch.cat(predicted, dim=1)
        # Mask target entries that exceed sequence lengths
        predicted = predicted * mask.float()
        # We only need last one for our case
        return p

    

In [40]:
class SelfAttentionLSTM(nn.Module):
    def __init__(self, word_embed_size=n_features, window_embed_size=128,
                 device=torch.device('cuda:0')):
        super(SelfAttentionLSTM, self).__init__()

        self.SelfAttention = NLPTransformer(word_embed_size)
        # self.dropout = nn.Dropout(p=0.1)
        # Store module in specified device (CUDA/CPU)
        self.device = (device if torch.cuda.is_available() else
                       torch.device('cpu'))
        self.to(self.device)

    def forward(self, inputs, length, mask=None):
        '''
        inputs = (batch_size=1, input_size, input_dimension)
        '''
        predict = self.SelfAttention(inputs, mask, length)
        return predict

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import pearsonr
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

model = SelfAttentionLSTM()
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)

def trainTransformer(category_tensor, line_tensor):
    
    model.zero_grad()
    
    # TODO: construct mask and lengths for batch size == 1
    lengths = torch.tensor([line_tensor.size()[0]])
    mask = torch.zeros(line_tensor.size()[0], line_tensor.size()[1], 1, dtype=torch.float)
    device = (torch.device('cuda') if torch.cuda.is_available() else
                   torch.device('cpu'))
    mask = mask.to(device)
    line_tensor = line_tensor.to(device)
    category_tensor = category_tensor.to(device)
    # END TODO
#     print(line_tensor.size())
#     print(category_tensor.size())
#     print(lengths.size())
#     print(mask.size())
    output = model(line_tensor, lengths, mask)
    # print(category_tensor.size())
    loss = criterion(output, category_tensor)
    loss.backward()

    # Add parameters' gradients to their values, multiplied by learning rate
    for p in model.parameters():
        if p.grad is None:
          continue
        p.data.add_(-learning_rate, p.grad.data)

    return output, loss.item()

In [ ]:
def categoryFromOutput(output):
  top_n,top_i = output.topk(1)
  category_i = top_i[0].item()
  return ['nothing','easy','medium','hard'][category_i],category_i
#print(output.topk(1))
#print(categoryFromOutput(output))

In [ ]:
def save_model(rnn, criterion, epoch, loss, filename):
  torch.save({
    'epoch': epoch,
    'model_state_dict': rnn.state_dict(),
    'optimizer_state_dict': criterion.state_dict(),
    'loss': loss,
  }, filename)

#print(criterion.state_dict())

In [ ]:
# rnn = RNN3(n_features, n_hidden, n_categories, dropout=0.2).cuda()

#learning_rate = 0.005 # If you set this too high, it might explode. If too low, it might not learn
learning_rate = 0.005 # If you set this too high, it might explode. If too low, it might not learn

def train2(category_tensor, line_tensor):
    hidden = rnn.initHidden()
    hidden = repackage_hidden(hidden)
    #hidden = hidden.cuda()

    rnn.zero_grad()

    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)

    loss = criterion(output, category_tensor)
    loss.backward()

    # Add parameters' gradients to their values, multiplied by learning rate
    for p in rnn.parameters():
        if p.grad is None:
          continue
        p.data.add_(-learning_rate, p.grad.data)

    return output, loss.item()

def repackage_hidden(h):
    """Wraps hidden states in new Tensors, to detach them from their history."""
    if isinstance(h, torch.Tensor):
        return h.detach()
    else:
        return tuple(repackage_hidden(v) for v in h)

n_iters = 100
print_every = 1
plot_every = 1
all_losses = []

model.train()
for epoch in range(1, n_iters + 1):
  all_training_items = iterateTrainingData()
  print('iteration', iter)
  current_loss = 0
  for idx,training_item in enumerate(all_training_items):
    (category,category_tensor, line_tensor) = training_item
    #print(line_tensor.size())
    if line_tensor.size()[0] == 0:
      continue
    output, loss = trainTransformer(category_tensor, line_tensor.permute(1,0,2))
    current_loss += loss
    if idx % 1000 == 0:
      #pass
      print(epoch, ':', idx, '/', len(all_training_items))
      # Print iter number, loss, name and guess
    if epoch % print_every == 0:
        guess, guess_i = categoryFromOutput(output)
        correct = '✓' if guess == category else '✗ (%s)' % category
        #print('%d %d%% (%s) %.4f / %s %s' % (iter, iter / n_iters * 100, timeSince(start), loss, guess, correct))

    # Add current loss avg to list of losses
    #if iter % plot_every == 0:
    #    all_losses.append(current_loss / plot_every)
    #    #current_loss = 0
  #rnn.zero_grad()
  all_losses.append(current_loss / plot_every)
  save_model(model, criterion, epoch, current_loss, 'model_attention_nohistory_nhidden512_v9_epoch' + str(epoch) + '.pt')

iteration <built-in function iter>
1 : 0 / 87994
1 : 1000 / 87994
1 : 2000 / 87994
1 : 3000 / 87994
1 : 4000 / 87994
1 : 5000 / 87994
1 : 6000 / 87994
1 : 7000 / 87994
1 : 8000 / 87994
1 : 9000 / 87994
1 : 10000 / 87994
1 : 11000 / 87994
1 : 12000 / 87994
1 : 13000 / 87994
1 : 14000 / 87994
1 : 15000 / 87994
1 : 16000 / 87994
1 : 17000 / 87994
1 : 18000 / 87994
1 : 19000 / 87994
1 : 20000 / 87994
1 : 21000 / 87994
1 : 22000 / 87994
1 : 23000 / 87994
1 : 24000 / 87994
1 : 25000 / 87994
1 : 26000 / 87994
1 : 27000 / 87994
1 : 28000 / 87994
1 : 29000 / 87994
1 : 30000 / 87994
1 : 31000 / 87994
1 : 32000 / 87994
1 : 33000 / 87994
1 : 34000 / 87994
1 : 35000 / 87994
1 : 36000 / 87994
1 : 37000 / 87994
1 : 38000 / 87994
1 : 39000 / 87994
1 : 40000 / 87994
1 : 41000 / 87994
1 : 42000 / 87994
1 : 43000 / 87994
1 : 44000 / 87994
1 : 45000 / 87994
1 : 46000 / 87994
1 : 47000 / 87994
1 : 48000 / 87994
1 : 49000 / 87994
1 : 50000 / 87994
1 : 51000 / 87994
1 : 52000 / 87994
1 : 53000 / 87994
1 : 54

In [ ]:
rnn = model

In [ ]:
rnn.train()
#all_training_items = iterateTrainingData()
for iter in range(1, n_iters + 1):
  print('iteration', iter)
  current_loss = 0
  for idx,training_item in enumerate(all_training_items):
    (category,category_tensor, line_tensor) = training_item
    #print(line_tensor.size())
    if line_tensor.size()[0] == 0:
      continue
    output, loss = train2(category_tensor.cuda(), line_tensor.cuda())
    current_loss += loss
    if idx % 1000 == 0:
      #pass
      print(iter, ':', idx, '/', len(all_training_items))
      # Print iter number, loss, name and guess
    if iter % print_every == 0:
        guess, guess_i = categoryFromOutput(output)
        correct = '✓' if guess == category else '✗ (%s)' % category
        #print('%d %d%% (%s) %.4f / %s %s' % (iter, iter / n_iters * 100, timeSince(start), loss, guess, correct))

    # Add current loss avg to list of losses
    #if iter % plot_every == 0:
    #    all_losses.append(current_loss / plot_every)
    #    #current_loss = 0
  #rnn.zero_grad()
  all_losses.append(current_loss / plot_every)

In [ ]:
import time
import math
import random

n_iters = 100
print_every = 1
plot_every = 1



# Keep track of losses for plotting
current_loss = 0
all_losses = []

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

start = time.time()

for iter in range(1, n_iters + 1):
  #category,category_tensor, line_tensor = randomTrainingExample()
  #if True:
  all_training_items = iterateTrainingData()
  for idx,training_item in enumerate(all_training_items):
    (category,category_tensor, line_tensor) = training_item
    if line_tensor.size()[0] == 0:
      continue
    output, loss = train(category_tensor.cuda(), line_tensor.cuda())
    current_loss += loss
    if idx % 1000 == 0:
      print(iter, ':', idx, '/', len(all_training_items))

  # Print iter number, loss, name and guess
  if iter % print_every == 0:
      guess, guess_i = categoryFromOutput(output)
      correct = '✓' if guess == category else '✗ (%s)' % category
      print('%d %d%% (%s) %.4f / %s %s' % (iter, iter / n_iters * 100, timeSince(start), loss, guess, correct))

  # Add current loss avg to list of losses
  if iter % plot_every == 0:
      all_losses.append(current_loss / plot_every)
      current_loss = 0

# for iter in range(1, n_iters + 1):
#     category,category_tensor, line_tensor = randomTrainingExample()
#     if line_tensor.size()[0] == 0:
#       continue
#     output, loss = train(category_tensor.cuda(), line_tensor.cuda())
#     current_loss += loss

#     # Print iter number, loss, name and guess
#     if iter % print_every == 0:
#         guess, guess_i = categoryFromOutput(output)
#         correct = '✓' if guess == category else '✗ (%s)' % category
#         print('%d %d%% (%s) %.4f / %s %s' % (iter, iter / n_iters * 100, timeSince(start), loss, guess, correct))

#     # Add current loss avg to list of losses
#     if iter % plot_every == 0:
#         all_losses.append(current_loss / plot_every)
#         current_loss = 0

In [ ]:
%matplotlib inline


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

plt.figure()
#plt.plot(all_losses[:10])
plt.plot(all_losses)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

plt.figure()
#plt.plot(all_losses[:10])
plt.plot(all_losses)

In [ ]:
all_categories = ['nothing', 'easy', 'medium', 'hard']

In [ ]:
def evaluate3(model, line_tensor):
    line_tensor = line_tensor.permute(1,0,2)
    #model.zero_grad()
    
    # TODO: construct mask and lengths for batch size == 1
    lengths = torch.tensor([line_tensor.size()[0]])
    mask = torch.zeros(line_tensor.size()[0], line_tensor.size()[1], 1, dtype=torch.float)
    device = (torch.device('cuda') if torch.cuda.is_available() else
                   torch.device('cpu'))
    mask = mask.to(device)
    line_tensor = line_tensor.to(device)
    #category_tensor = category_tensor.to(device)
    # END TODO
#     print(line_tensor.size())
#     print(category_tensor.size())
#     print(lengths.size())
#     print(mask.size())
    output = model(line_tensor, lengths, mask)
    # print(category_tensor.size())
    #loss = criterion(output, category_tensor)
    #loss.backward()
    return output

In [ ]:
def evaluate2(line_tensor):
    line_tensor = line_tensor.permute(1,0,2)
    #model.zero_grad()
    
    # TODO: construct mask and lengths for batch size == 1
    lengths = torch.tensor([line_tensor.size()[0]])
    mask = torch.zeros(line_tensor.size()[0], line_tensor.size()[1], 1, dtype=torch.float)
    device = (torch.device('cuda') if torch.cuda.is_available() else
                   torch.device('cpu'))
    mask = mask.to(device)
    line_tensor = line_tensor.to(device)
    #category_tensor = category_tensor.to(device)
    # END TODO
#     print(line_tensor.size())
#     print(category_tensor.size())
#     print(lengths.size())
#     print(mask.size())
    output = model(line_tensor, lengths, mask)
    # print(category_tensor.size())
    #loss = criterion(output, category_tensor)
    #loss.backward()
    return output

    # Add parameters' gradients to their values, multiplied by learning rate
#     for p in model.parameters():
#         if p.grad is None:
#           continue
#         p.data.add_(-learning_rate, p.grad.data)

#     return output, loss.item()

In [ ]:
# Just return an output given a line
def evaluate(line_tensor):
    rnn.eval()
    #hidden = rnn.initHidden() #.cuda()

    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)
    
    return output

In [ ]:
# Keep track of correct guesses in a confusion matrix
confusion = torch.zeros(n_categories, n_categories)
n_confusion = 10000



# Go through a bunch of examples and record which are correctly guessed
for i in range(n_confusion):
    category, category_tensor, line_tensor = randomTrainingExample()
    if line_tensor.size()[0] == 0:
      continue
    output = evaluate(line_tensor.cuda())
    guess, guess_i = categoryFromOutput(output)
    category_i = difficulty_to_idx[category]
    confusion[category_i][guess_i] += 1

# Normalize by dividing every row by its sum
for i in range(n_categories):
    confusion[i] = confusion[i] / confusion[i].sum()

# Set up plot
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(confusion.numpy())
fig.colorbar(cax)

# Set up axes
ax.set_xticklabels([''] + all_categories, rotation=90)
ax.set_yticklabels([''] + all_categories)

# Force label at every tick
ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

# sphinx_gallery_thumbnail_number = 2
plt.show()

In [ ]:
#print(dev_data[0])

In [ ]:
def evaluate_model_on_dataset(model, dataset):
  num_correct = 0
  num_incorrect = 0
  for item in dataset:
    category_tensor = item['category']
    feature_tensor = item['feature']
    if feature_tensor.size()[0] == 0:
        continue
    difficulty = tensor_to_difficulty(category_tensor)
    difficulty_idx = difficulty_to_idx[difficulty]
    predicted_tensor = evaluate3(model, feature_tensor.cuda())
    predicted_difficulty,predicted_difficulty_idx = categoryFromOutput(predicted_tensor)
    if predicted_difficulty_idx == difficulty_idx:
      num_correct += 1
    else:
      num_incorrect += 1
    confusion[difficulty_idx][predicted_difficulty_idx] += 1
    #print(predicted_difficulty)
    #print(difficulty)
    #break

  print('num correct: ' + str(num_correct))
  print('num incorrect: ' + str(num_incorrect))
  print('percentage correct: ' + str(num_correct / (num_correct + num_incorrect)))

In [ ]:
import os
import os.path

In [1]:
epoch = 0
while True:
  epoch += 1
  model_filename = 'model_attention_nohistory_nhidden512_v9_epoch' + str(epoch) + '.pt'
  if not os.path.isfile(model_filename):
    break
  print(model_filename)
  model = SelfAttentionLSTM()
  model_state = torch.load(model_filename)
  model.load_state_dict(model_state['model_state_dict'])
  model.eval()
  evaluate_model_on_dataset(model, dev_data)
  #break


NameError: name 'os' is not defined

In [ ]:
epoch = 0
while True:
  epoch += 1
  model_filename = 'model_attention_nohistory_v8_epoch' + str(epoch) + '.pt'
  if not os.path.isfile(model_filename):
    break
  print(model_filename)
  model = SelfAttentionLSTM()
  model_state = torch.load(model_filename)
  model.load_state_dict(model_state['model_state_dict'])
  model.eval()
  evaluate_model_on_dataset(model, training_data)
  #break


In [ ]:
#confusion = torch.zeros(n_categories, n_categories)
num_correct = 0
num_incorrect = 0
confusion = torch.zeros(n_categories, n_categories)

for item in training_data:
  category_tensor = item['category']
  feature_tensor = item['feature']
  if feature_tensor.size()[0] == 0:
      continue
  difficulty = tensor_to_difficulty(category_tensor)
  difficulty_idx = difficulty_to_idx[difficulty]
  predicted_tensor = evaluate2(feature_tensor.cuda())
  predicted_difficulty,predicted_difficulty_idx = categoryFromOutput(predicted_tensor)
  if predicted_difficulty_idx == difficulty_idx:
    num_correct += 1
  else:
    num_incorrect += 1
  confusion[difficulty_idx][predicted_difficulty_idx] += 1
  #print(predicted_difficulty)
  #print(difficulty)
  #break

print('num correct: ' + str(num_correct))
print('num incorrect: ' + str(num_incorrect))
print('percentage correct: ' + str(num_correct / (num_correct + num_incorrect)))

In [ ]:
#confusion = torch.zeros(n_categories, n_categories)
num_correct = 0
num_incorrect = 0
confusion = torch.zeros(n_categories, n_categories)

for item in dev_data:
  category_tensor = item['category']
  feature_tensor = item['feature']
  if feature_tensor.size()[0] == 0:
      continue
  difficulty = tensor_to_difficulty(category_tensor)
  difficulty_idx = difficulty_to_idx[difficulty]
  predicted_tensor = evaluate2(feature_tensor.cuda())
  predicted_difficulty,predicted_difficulty_idx = categoryFromOutput(predicted_tensor)
  if predicted_difficulty_idx == difficulty_idx:
    num_correct += 1
  else:
    num_incorrect += 1
  confusion[difficulty_idx][predicted_difficulty_idx] += 1
  #print(predicted_difficulty)
  #print(difficulty)
  #break

print('num correct: ' + str(num_correct))
print('num incorrect: ' + str(num_incorrect))
print('percentage correct: ' + str(num_correct / (num_correct + num_incorrect)))

In [ ]:
# Normalize by dividing every row by its sum
for i in range(n_categories):
    confusion[i] = confusion[i] / confusion[i].sum()

# Set up plot
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(confusion.numpy())
fig.colorbar(cax)

# Set up axes
ax.set_xticklabels([''] + all_categories, rotation=90)
ax.set_yticklabels([''] + all_categories)

# Force label at every tick
ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

# sphinx_gallery_thumbnail_number = 2
plt.show()

In [ ]:
#confusion = torch.zeros(n_categories, n_categories)
num_correct = 0
num_incorrect = 0
confusion = torch.zeros(n_categories, n_categories)

for item in dev_data:
  user = item['user']
  category_tensor = item['category']
  feature_tensor = item['feature']
  if feature_tensor.size()[0] == 0:
      continue
  difficulty = tensor_to_difficulty(category_tensor)
  difficulty_idx = difficulty_to_idx[difficulty]
  predicted_difficulty = get_most_common_difficulty_for_user(user)
  predicted_difficulty_idx = difficulty_to_idx[predicted_difficulty]
  predicted_tensor = make_tensor_from_chosen_difficulty(predicted_difficulty)
  #predicted_tensor = evaluate(feature_tensor.cuda())
  #predicted_difficulty,predicted_difficulty_idx = categoryFromOutput(predicted_tensor)
  if predicted_difficulty_idx == difficulty_idx:
    num_correct += 1
  else:
    num_incorrect += 1
  confusion[difficulty_idx][predicted_difficulty_idx] += 1
  #print(predicted_difficulty)
  #print(difficulty)
  #break

print('num correct: ' + str(num_correct))
print('num incorrect: ' + str(num_incorrect))
print('percentage correct: ' + str(num_correct / (num_correct + num_incorrect)))

In [ ]:
def get_most_recent_difficulty(tensor):
  difficulty_arr = list(tensor[-1][0].data.cpu().numpy())
  max_idx = difficulty_arr.index(1)
  return ['nothing', 'easy', 'medium', 'hard'][max_idx]

#confusion = torch.zeros(n_categories, n_categories)
num_correct = 0
num_incorrect = 0
confusion = torch.zeros(n_categories, n_categories)

for item in dev_data:
  user = item['user']
  category_tensor = item['category']
  feature_tensor = item['feature']
  if feature_tensor.size()[0] == 0:
      continue
  difficulty = tensor_to_difficulty(category_tensor)
  difficulty_idx = difficulty_to_idx[difficulty]
  predicted_difficulty = get_most_recent_difficulty(feature_tensor)
  predicted_difficulty_idx = difficulty_to_idx[predicted_difficulty]
  predicted_tensor = make_tensor_from_chosen_difficulty(predicted_difficulty)
  #predicted_tensor = evaluate(feature_tensor.cuda())
  #predicted_difficulty,predicted_difficulty_idx = categoryFromOutput(predicted_tensor)
  if predicted_difficulty_idx == difficulty_idx:
    num_correct += 1
  else:
    num_incorrect += 1
  confusion[difficulty_idx][predicted_difficulty_idx] += 1
  #print(predicted_difficulty)
  #print(difficulty)
  #break

print('num correct: ' + str(num_correct))
print('num incorrect: ' + str(num_incorrect))
print('percentage correct: ' + str(num_correct / (num_correct + num_incorrect)))

In [ ]:
#confusion = torch.zeros(n_categories, n_categories)
num_correct = 0
num_incorrect = 0
confusion = torch.zeros(n_categories, n_categories)

for item in dev_data:
  user = item['user']
  category_tensor = item['category']
  feature_tensor = item['feature']
  if feature_tensor.size()[0] == 0:
      continue
  difficulty = tensor_to_difficulty(category_tensor)
  difficulty_idx = difficulty_to_idx[difficulty]
  #predicted_difficulty = get_most_common_difficulty_for_user(user)
  predicted_difficulty = get_most_common_difficulty_overall()
  predicted_difficulty_idx = difficulty_to_idx[predicted_difficulty]
  predicted_tensor = make_tensor_from_chosen_difficulty(predicted_difficulty)
  #predicted_tensor = evaluate(feature_tensor.cuda())
  #predicted_difficulty,predicted_difficulty_idx = categoryFromOutput(predicted_tensor)
  if predicted_difficulty_idx == difficulty_idx:
    num_correct += 1
  else:
    num_incorrect += 1
  confusion[difficulty_idx][predicted_difficulty_idx] += 1
  #print(predicted_difficulty)
  #print(difficulty)
  #break

print('num correct: ' + str(num_correct))
print('num incorrect: ' + str(num_incorrect))
print('percentage correct: ' + str(num_correct / (num_correct + num_incorrect)))